In [21]:
import os
import re
import pandas as pd
from tqdm import tqdm
from ekonlpy.sentiment import MPCK
import glob
import csv

In [22]:
mpck = MPCK()

def clean_meetings(text):
    start_pattern = r'2\s+본회의에서는'
    end_pattern = r'4\s+(정부측|한국은행|의결사항|토의결론|심의결과|기준금리)'

    start_match = re.search(start_pattern, text)
    end_match = re.search(end_pattern, text)
    if start_match:
        content = text[start_match.end() : end_match.start()] if end_match else text[start_match.end() :]
    else:
        content = text
    content = re.sub(r'전일 개최된.*?보고하였음', '', content)
    content = re.sub(r'전일 개최된.*?토의가 있었음\.', ' ', content)
    content = re.sub(r'\d?\s?본회의에서는 의장이.*?접수하였음', '', content)
    content = re.sub(r'위원 전원 찬성으로 가결하였음', '', content)
    content = re.sub(r'만장일치로.*?결정함', '', content)
    content = re.sub(r'의결사항.*?운용하기로 하였다', '', content)
    
    intro_noises = [
        r'국내외\s+경제동향\s+과\s+관련하여\s+(먼저)?', # "국내외 경제동향 과 관련하여 먼저"
        r"외환\s+국제금융\s+동향\s*과\s+관련하여",
        r"금융시장\s+동향\s*과\s+관련하여",
        r'일부\s+위원(은|들은)', # "일부 위원은", "일부 위원들은"
        r'이에\s+대해\s+관련\s+부서는', # "이에 대해 관련 부서는"
        r'이어서\s+동\s+위원은', # "이어서 동 위원은"
        r'또한\s+동\s+위원은', # "또한 동 위원은"
        r'아울러\s+동\s+위원은', # "아울러 동 위원은"
        r'다른\s+일부\s+위원은',
        r'또\s+다른\s+일부\s+위원은',
        r'질의하였음', r'답변하였음', r'언급하였음', r'견해를\s+밝혔음', r'당부하였음', # 서술어들
        r"의장이\s+한국은행법.*?상정하고.*?보고\s*내용.*?접수하였음",
        r"3\s+위원\s+토의내용",
        r"보고\s+내용\s+별첨",
        r"이에\s+대해\s+관련부서에서는"
    ]
    
    for noise in intro_noises:
        content = re.sub(noise, '', content)

    noise_keywords = [ '보고 내용 별첨', '별 첨', '통화정책방향 관련 보고내용', '의안 제\d+호', '제\d+호' ]
    for word in noise_keywords: content = re.sub(word, '', content)
    
    content = re.sub(r'\s\d{1,2}\s', ' ', content)
    content = re.sub(r'[^가-힣0-9\s\.\,\%]', '', content)
    content = re.sub(r'\s+', ' ', content).strip()
    return content

def ngramize(tokens, max_n=5):
    keep_tags = ['NNG', 'VV', 'VA', 'MAG']
    filtered = [w for w in tokens if w.split('/')[1] in keep_tags]
    ngram_results = []
    for pos in range(len(filtered)):
        for n in range(1, max_n + 1):
            if pos + n <= len(filtered):
                ngram = ";".join(filtered[pos : pos + n])
                ngram_results.append(ngram)
                
    return ngram_results

def extract_date(filename):
    match = re.search(r'\d{8}', filename)
    if match:
        d = match.group()
        return f"{d[:4]}-{d[4:6]}-{d[6:]}"
    return "0000-00-00"

In [23]:
folder_path = './final_txt/*.txt'
output_path = "meeting_tokens.csv"
results = []

file_list = glob.glob(folder_path)
print(f"📂 총 {len(file_list)}개의 의사록 파일을 찾았습니다.")


for file_path in tqdm(file_list, desc="의사록 처리 중"):    
    try:
        with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
            raw_text = f.read()
        
        file_name = os.path.basename(file_path)
        date = extract_date(file_name)

        cleaned_text = clean_meetings(raw_text)
        
        tokens = mpck.tokenize(cleaned_text)
        
        # 3. n-gram
        ngrams = ngramize(tokens)
        
        # 4. 결과 저장
        results.append({
            'date': date,
            'content': cleaned_text,
            'tokens': ', '.join(ngrams),
            'category': '의사록', 
            'source': '한국은행'
        })
        
    except Exception as e:
        print(f"❌ {file_path} 처리 중 에러: {e}")

df_final = pd.DataFrame(results)
df_final.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"✨ 모든 작업 완료! 결과: {output_path}")

📂 총 150개의 의사록 파일을 찾았습니다.


의사록 처리 중: 100%|██████████| 150/150 [00:24<00:00,  6.19it/s]


✨ 모든 작업 완료! 결과: meeting_tokens.csv


In [24]:
import pandas as pd
import json
import csv

df_meeting = pd.read_csv('meeting_tokens.csv', encoding='utf-8-sig')

def string_to_list(text):
    if pd.isna(text): return "[]" # 빈 칸 처리
    clean_list = [t.strip() for t in str(text).split(',') if t.strip()]
    return json.dumps(clean_list, ensure_ascii=False)

df_meeting['tokens'] = df_meeting['tokens'].apply(string_to_list)

output_path = 'meeting_preprocessed_fixed.csv'
df_meeting.to_csv(output_path, index=False, encoding='utf-8-sig', quoting=csv.QUOTE_ALL)

In [25]:
df_meeting.head(30)

,date,content,tokens,category,source
0,2012-01-13,최근 건설투자 등 일부 경제지표 가 지난해 12월 예상했던 것보다 부진한 것으로 나...,"[""건설/NNG"", ""건설/NNG;투자/NNG"", ""건설/NNG;투자/NNG;경제/...",의사록,한국은행
1,2012-02-09,최근 국내경제의 성장 모멘 텀이 다소 약화되고 있으며 특히 소비의 경우에는 소비심리...,"[""경제/NNG"", ""경제/NNG;성장/NNG"", ""경제/NNG;성장/NNG;모멘텀...",의사록,한국은행
2,2012-03-08,최근의 선진국 및 개도국 경제상황 변화와 관련하여 우리나라의 4분기 경기상황에 대한...,"[""선진국/NNG"", ""선진국/NNG;신흥국/NNG"", ""선진국/NNG;신흥국/NN...",의사록,한국은행
3,2012-04-13,금년중 물가가 지난번 전망 시에는 상고하저를 나타낼 것으로 보았으나 이번 수정 전망...,"[""중/NNG"", ""중/NNG;물가/NNG"", ""중/NNG;물가/NNG;지난번/NN...",의사록,한국은행
4,2012-05-10,그동안 금융통화위원회가 유지 해왔던 금리운용의 스탠스를 물가 경기 등의 측면에서 다...,"[""유지/NNG"", ""유지/NNG;금리/NNG"", ""유지/NNG;금리/NNG;운용/...",의사록,한국은행
5,2012-06-08,현재 당행의 올해 경기전망은 상반기 중에 유로지역의 재정위기가 어느 정도 진정될 것...,"[""경기/NNG"", ""경기/NNG;전망/NNG"", ""경기/NNG;전망/NNG;중/N...",의사록,한국은행
6,2012-07-12,하반기중 경기가 다소 호전될 것으로 전망하고 있지만 최근의 유로사태 전개양상이 지난...,"[""중/NNG"", ""중/NNG;경기/NNG"", ""중/NNG;경기/NNG;다소/MAG...",의사록,한국은행
7,2012-08-09,그동안 대외여건 악화에도 불구 하고 우리경제 견인의 핵심요인이었던 수출 모멘텀이 약...,"[""대외/NNG"", ""대외/NNG;여건/NNG"", ""대외/NNG;여건/NNG;악화/...",의사록,한국은행
8,2012-09-13,금년중 물가가 당초 예상보다 낮은 수준을 보일 것으로 예상되지만 그 보다는 향후 통...,"[""중/NNG"", ""중/NNG;물가/NNG"", ""중/NNG;물가/NNG;예상/NNG...",의사록,한국은행
9,2012-10-11,이번 달 수정전망에서 금년도 성 장률 전망치를 7월 전망시보다 큰 폭 하향조정한 이...,"[""달/NNG"", ""달/NNG;수정/NNG"", ""달/NNG;수정/NNG;전망/NNG...",의사록,한국은행
